In [ ]:
import requests
import json
import pandas as pd
import base64
import ipywidgets as widgets
from ipywidgets import Layout
from PIL import Image
import numpy as np
import rioxarray as rxr
import xarray as xr
from urllib.request import urlopen
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
import holoviews.operation.datashader as hd
import bokeh as bk
from holoviews import opts
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2
import WEkEO_methods as m

In [ ]:
# import hda
# Initialize HDA client
#c = hda.Client()  #upload .hdarc file in the folder

- Reference: https://www.wekeo.eu/docs/harmonised-data-access-api
- API: https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/ui/#/

## Get token

Reference used to get the access token follow this page instructions: 
- https://curlconverter.com/ to translate the curl request
- https://www.base64encode.org/ write "username:password" and copy&paste the base64encore in the headers below.

The following widget allows to login with your WEkEO credentials and obtain the token with the **get_token** function.

In [ ]:
username = m.text_widget("Insert username: ")
password = m.password_widget("Insert password: ")

login_box = widgets.VBox([username, password])
login_box

In [ ]:
headers = m.get_token(username, password)
token = list(headers.values())[0]

----

## ERA5 - Reanalysis Single Levels

In [ ]:
# dataset_id = "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS"

In [ ]:
# df = m.get_all_data_request(2000)
# dataset_list = df["datasetId"]

In [ ]:
name = m.text_widget("Insert data name: ")
name

Select "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS" from the Dropdown list:

In [ ]:
data_df = m.get_data_from_name_request(2000, name.value)
dataset_list = data_df["datasetId"]
dataset_id = m.get_dropdown(dataset_list, "List of datasets: ")
dataset_id

In [ ]:
description = m.get_description(data_df, dataset_id)

In [ ]:
image = m.display_image(data_df, dataset_id, 500, "jpg", 600 )
image

In [ ]:
metadata = m.get_metadata(dataset_id, headers)

In [ ]:
variables_list = m.era5_single_levels_list(metadata)

In [ ]:
params_sel = m.select_multiple(variables_list, "Variable: ")

In [ ]:
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
year_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
month_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
day_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
download_list = ["Download NETCDF", "Read NETCDF in memory"]

In [ ]:
product_type_sel = m.select_multiple(product_type_list, "Product type:")
year_sel = m.select_multiple(year_list, "Year: ")
month_sel = m.select_multiple(month_list, "Month: ")
day_sel = m.select_multiple(day_list, "Day: ")
time_sel = m.select_multiple(time_list, "Time: ")
format_type_sel = m.select_buttons(format_type_list, "Format: ", "netcdf")
download_sel = m.select_buttons(download_list, "Data download: ", "Read NETCDF in memory")

In [ ]:
title= widgets.HTML('<h2 style="text-align:center;">ERA5 - Reanalysis Single Levels</h2><hr><h3 style="padding: 10px;">Input variables - Multiple selection</h3>')
title2 = widgets.HTML('<details><h3>'+str(description[0])+'</div><h3></details>')
variables = widgets.VBox([title, params_sel, product_type_sel, year_sel, month_sel, day_sel, time_sel, format_type_sel, download_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(header=title2,
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

----

In [ ]:
job = m.api_query_era5_single_levels(dataset_id, params_sel, year_sel, month_sel, day_sel, time_sel, product_type_sel, format_type_sel, token)

In [ ]:
jobId = job['jobId']
print("The job ID is: "+jobId)

In [ ]:
url = m.request_data(jobId, token)

In [ ]:
save_as = m.text_widget("Save file as (only if you want to download): ")
save_as

In [ ]:
ds = m.download_type(download_sel, download_list, url, save_as)

In [ ]:
ds

In [ ]:
variables = list(ds.keys())
var_drop = widgets.Dropdown(
    options=variables,
    description='Variable:',
    disabled=False,
    width = '50px'
)

var_drop

In [ ]:
ds_data = ds[var_drop.value].squeeze()
ds_data

In [ ]:
# f = plt.figure(figsize=(10,8))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
# gl= ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
# gl.right_labels=False
# gl.top_labels=False
# ax.add_feature(cfeature.LAND, zorder=1,edgecolor='k')
# ax.set_extent([-180,180, -90, 90],crs=ccrs.PlateCarree())

# f.suptitle(ds_data.long_name, fontsize=28)
# im = ax.pcolor(ds_data['longitude'].data,ds_data['latitude'].data, ds_data, cmap='plasma')
# cbar= f.colorbar(im,ax=ax)
# cbar.set_label(ds_data.units, rotation =270, fontsize=23,labelpad=15)

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom'],
                title=ds_data.long_name+" ["+ds_data.units+"]"),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
ds = ds_data.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')

hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')

hv.element.tiles.OSM() * hv_image_basic

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(-90, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

----

# CAMS

In [ ]:
dataset_id = "EO:ECMWF:DAT:CAMS_EUROPE_AIR_QUALITY_FORECASTS"

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size=2000")
dataset_text = dataset.text
data = json.loads(dataset_text)

In [ ]:
# from JSON to Pandas DataFrame
data_df = pd.json_normalize(data['content'])
data_df

In [ ]:
abstract = data_df.loc[data_df['datasetId'] == dataset_id]
description = list(abstract["abstract"])

In [ ]:
img_url = list(abstract["previewImage"])[0]
im = Image.open(requests.get(img_url, stream=True).raw)

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+dataset_id, headers=headers)
dataset_text = dataset.text
metadata = json.loads(dataset_text)

In [ ]:
abstract = data_df.loc[data_df["datasetId"] == dataset_id]

In [ ]:
params_list = list(metadata['parameters']['multiStringSelects'][0]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
product_type_list = list(metadata['parameters']['multiStringSelects'][1]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
level_list = list(metadata['parameters']['multiStringSelects'][2]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
type_model_list = list(metadata['parameters']['multiStringSelects'][3]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
time_list = list(metadata['parameters']['multiStringSelects'][4]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
leadtime_list = list(metadata['parameters']['multiStringSelects'][5]['details']['groupedValueLabels'][0]['valuesLabels'].keys())
format_type_list = list(metadata['parameters']['stringChoices'][0]['details']['valuesLabels'].keys())

In [ ]:
params_sel = widgets.SelectMultiple(
    options=params_list,
    description='Select parameters:',
    disabled=False)

In [ ]:
product_type_sel = widgets.SelectMultiple(
    options=product_type_list,
    description='Product type:',
    disabled=False)

In [ ]:
level_sel = widgets.SelectMultiple(
    options=level_list,
    description='Level:',
    disabled=False)

In [ ]:
type_model_sel = widgets.SelectMultiple(
    options=type_model_list,
    description='Model: ',
    disabled=False)

In [ ]:
time_sel = widgets.SelectMultiple(
    options=time_list,
    description='Time: ',
    disabled=False)

In [ ]:
leadtime_sel = widgets.SelectMultiple(
    options=leadtime_list,
    description='Leadtime: ',
    disabled=False)

In [ ]:
format_type_sel = widgets.RadioButtons(
    options=format_type_list,
    value='netcdf',
    layout={'width': 'max-content'}, # If the items' names are long
    description='Format type (netcdf for xarray):',
        disabled=False)

In [ ]:
start_date = widgets.DatePicker(
    description='Pick starting date: ',
    disabled=False
)

In [ ]:
end_date = widgets.DatePicker(
    description='Pick ending date: ',
    disabled=False
)

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, LayersControl

satellite = basemap_to_tiles(basemaps.Gaode.Satellite)
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

m = Map(layers=(satellite, osm ), center=(45, 10), zoom=4)

dc = DrawControl()
lc = LayersControl(position='topright')

dc = DrawControl(
    marker={"shapeOptions": {"color": "#0000FF"}},
    rectangle={"shapeOptions": {"color": "#0000FF"}},
    circle={"shapeOptions": {"color": "#0000FF"}},
    circlemarker={},
)

def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)


dc.on_draw(handle_draw)
m.add_control(dc)
m.add_control(lc)

m

In [ ]:
coords = dc.last_draw['geometry']['coordinates'][0]
coords

In [ ]:
W = coords[1][0]
E = coords[3][0]
N = coords[1][1]
S = coords[3][1]

In [ ]:
import IPython
image = IPython.display.Image(img_url, width = 300)

image = widgets.Image(
  value=image.data,
  format='jpg', 
  width=600,
  height=700,
)

In [ ]:
title= widgets.HTML('<h2 style="text-align:center;">CAMS European air quality forecasts </h2><hr><h3 style="padding: 10px;">Input variables - Multiple selection</h3>')
title2 = widgets.HTML('<details><h3>'+str(description[0])+'</div><h3></details>')
variables = widgets.VBox([title, params_sel, product_type_sel, level_sel, type_model_sel, leadtime_sel,time_sel, start_date, end_date, format_type_sel])
image_box = widgets.VBox([image])


ui = widgets.AppLayout(header=title2,
          left_sidebar=variables,
          right_sidebar=image_box,
          layout=widgets.Layout(border='solid'))

# compute_button.on_click(compute_button_f)
container = widgets.Box([ui])
display(container)

In [ ]:
query = {
  "datasetId": dataset_id,
  "boundingBoxValues": [
    {
      "name": "area",
      "bbox": [
        W,
        N,
        E,
        S
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "date",
      "start": start_date.value.strftime("%Y-%m-%dT%H:%M:%S.000Z"),
      "end": end_date.value.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    }
  ],
  "multiStringSelectValues": [
    {
      "name": "variable",
      "value": list(params_sel.value)
    },
    {
      "name": "model",
      "value": list(product_type_sel.value)
    },
    {
      "name": "level",
      "value": list(level_sel.value)
    },
    {
      "name": "type",
      "value": list(type_model_sel.value)
    },
    {
      "name": "time",
      "value": list(time_sel.value)

    },
    {
      "name": "leadtime_hour",
      "value": list(leadtime_sel.value)
    }
  ],
  "stringChoiceValues": [
    {
      "name": "format",
      "value": format_type_sel.value
    }
  ]
}
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'authorization': 'Basic '+str(token['access_token'])
}

data = json.dumps(query)
dataset_post = requests.post("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datarequest", headers=headers, data=data)
dataset_post_text = dataset_post.text
job_id = json.loads(dataset_post_text)

# # The following line runs the query
# matches = c.search(query)

print(dataset_post_text)

In [ ]:
jobId = job_id['jobId']
print("The job ID is: "+jobId)

In [ ]:
headers = {'authorization': 'Basic '+str(token['access_token'])}


status_request = requests.get('https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest/status/'+jobId, headers=headers)
status = status_request.text
status_message = json.loads(status)['status']
if status_message == "running":
    print("Download status: "+ status_message +"...")
if status_message == "completed":     
    print("Download status: "+ status_message +"  \u2713")
    
while status_message == "running":
    status_request = requests.get('https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest/status/'+jobId, headers=headers)
    status = status_request.text
    status_message = json.loads(status)['status']
    if status_message == "running":
        print("Download status: "+ status_message +"...", end='\r')
    if status_message == "completed":     
        print("Download status: "+ status_message +"  \u2713")

In [ ]:
headers = {'authorization': 'Basic '+str(token['access_token'])}
get_url_request = requests.get('https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest/jobs/'+jobId+'/result', headers=headers)
get_url = json.loads(get_url_request.text)
print('The URL for download is: '+ get_url['content'][0]['url'])

In [ ]:
fl = get_url['content'][0]['url']
# load into memory 
with urlopen(fl) as f:
    ds = xr.open_dataset(f.read())

In [ ]:
ds = ds.mean(dim='time')

In [ ]:
variables = list(ds.keys())
var_drop = widgets.Dropdown(
    options=variables,
    description='Variable:',
    disabled=False,
    width = '50px'
)

var_drop

In [ ]:
ds_data = ds[var_drop.value].squeeze()

In [ ]:
# f = plt.figure(figsize=(10,8))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
# gl= ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
# gl.right_labels=False
# gl.top_labels=False
# ax.add_feature(cfeature.LAND, zorder=1,edgecolor='k')
# ax.set_extent([-180,180, -90, 90],crs=ccrs.PlateCarree())

# f.suptitle(ds_data.name, fontsize=28)
# im = ax.pcolor(ds_data['longitude'].data,ds_data['latitude'].data, ds_data, cmap='plasma')
# cbar= f.colorbar(im,ax=ax)
# # cbar.set_label(ds_data.units, rotation =270, fontsize=23,labelpad=15)

In [ ]:
hv_dataset = hv.Dataset(ds_data)
hv_dataset.data

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom'],
                title=ds_data.name),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
# ds = ds_data.rio.write_crs('EPSG:4326')
# ds_mercator = ds.rio.reproject('EPSG:3857')

# hv_dataset = hv.Dataset(ds_mercator)
# hv_image_basic = hv.Image(hv_dataset)
# hv_image_basic = hv_image_basic.options(cmap='inferno')

# hv.element.tiles.OSM() * hv_image_basic

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, projection=ccrs.Orthographic(4, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

In [ ]:
ds.hvplot.quadmesh(
    'longitude', 'latitude', var_drop.value, crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    global_extent=True, frame_height=400, cmap='viridis',
    coastline=True
)

---

## Datasets exploration

In [ ]:
dataset = requests.get("https://wekeo-broker-k8s.apps.mercator.dpi.wekeo.eu/databroker/datasets?size=2000")
dataset_text = dataset.text
data = json.loads(dataset_text)

Dataframe containing all the datasets available on WEkEO, with an abstract, a preview image and a title:

In [ ]:
# from JSON to Pandas DataFrame
data_df = pd.json_normalize(data['content'])
data_df

Select the dataset id with a dropdown:

In [ ]:
dataset_list = data_df['datasetId']

datasetId_drop = widgets.Dropdown(
    options=list(dataset_list),
    description='Dataset ID:',
    disabled=False)

datasetId_drop

In [ ]:
datasetId_drop.value

Read abstract corresponding to the dataset ID selected:

In [ ]:
abstract = data_df.loc[data_df['datasetId'] == str(datasetId_drop.value)]

In [ ]:
print(list(abstract["abstract"]))

Plot a preview image of the selected dataset:

In [ ]:
img_url = list(abstract["previewImage"])[0]
im = Image.open(requests.get(img_url, stream=True).raw)
plt.imshow(im)

Request dataset metadata:

In [ ]:
dataset = requests.get("https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/querymetadata/"+str(datasetId_drop.value), headers=headers)
dataset_text = dataset.text
dataset = json.loads(dataset_text)

In [ ]:
dataset

Select dataset category:

In [ ]:
category = dataset['parameters']['multiStringSelects'][0]['details']['groupedValueLabels']
category_list = []
for item in category:
     category_list.append(item["label"])

category_drop = widgets.Dropdown(
    options=category_list,
    description='Category:',
    disabled=False)

category_drop

In [ ]:
category_list
category_n = list(range(len(category_list)))

In [ ]:
category_n

In [ ]:
category_dict = dict(zip(category_list, category_n))
category_dict

In [ ]:
parameter = category[category_dict[str(category_drop.value)]]
parameter

In [ ]:
params_list = list(parameter['valuesLabels'].keys())

In [ ]:
params_drop = widgets.Dropdown(
    options=params_list,
    description='Category:',
    disabled=False)

params_drop

In [ ]:
metadata['parameters']['multiStringSelects'][1]

In [ ]:
metadata

# TEST

In [ ]:
if query['stringChoiceValues'][0]['value'] == 'netcdf':
    fl = matches.results[0]['url']
    print('Reading NetCDF file into memory')
    # load into memory 
    with urlopen(fl) as f:
        ds = xr.open_dataset(f.read()).mean(dim='time')
elif query['stringChoiceValues'][0]['value'] == 'zip':
    print('Downloading zip file')
    matches.download()

In [ ]:
import glob, os

zip_list = []
os.chdir("./")
for file in glob.glob("*.zip"):
    print(file)
    zip_list.append(file)

In [ ]:
zip_list

In [ ]:
import ipywidgets as widgets

zip_drop = widgets.Dropdown(
    options=zip_list,
    description='Zip folder:',
    disabled=False,
)
zip_drop

In [ ]:
import zipfile
with zipfile.ZipFile(str(zip_drop.value), 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
nc_list = []
os.chdir("./")
for file in glob.glob("*.nc"):
    print(file)
    nc_list.append(file)

In [ ]:
nc_list

In [ ]:
nc_drop = widgets.Dropdown(
    options=nc_list,
    description='NetCDF folder:',
    disabled=False,
)
nc_drop

In [ ]:
ds = xr.open_dataset(nc_drop.value).mean(dim='time')

In [ ]:
hv_dataset = hv.Dataset(ds)
hv_dataset.data

In [ ]:
ds_data = ds['crwc'].squeeze()

In [ ]:
map_height=400
map_width=800

hv_tiles_osm = hv.element.tiles.OSM()

hv.opts.defaults(
  hv.opts.Image(alpha=0.7,
                height=map_height, width=map_width, 
                colorbar=True, 
                tools=['hover'], active_tools=['wheel_zoom']),
  hv.opts.Tiles(active_tools=['wheel_zoom'], height=map_height, width=map_width)
)

hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv_image_basic

In [ ]:
ds = ds.rio.write_crs('EPSG:4326')
ds_mercator = ds.rio.reproject('EPSG:3857')
hv_dataset = hv.Dataset(ds_mercator)
hv_image_basic = hv.Image(hv_dataset)
hv_image_basic = hv_image_basic.options(cmap='inferno')
hv.element.tiles.OSM() * hv_image_basic